import cudf
#from numba import cuda
import numpy as np
import pandas as pd
import json
from collections import OrderedDict

import cuspatial
import geopandas as gpd

Dataset is available at: 

In [1]:
import cudf
import cuspatial

import numpy as np
import geopandas as gpd

## Reading Points with CUDF

In [2]:
%%time
parking = cudf.read_csv("parking_violations.csv")

CPU times: user 1.67 s, sys: 973 ms, total: 2.64 s
Wall time: 4.29 s


In [3]:
parking.shape

(8347724, 13)

In [4]:
parking.head()

,anon_ticket_number,issue_datetime,state,anon_plate_id,division,location,violation_desc,fine,issuing_agency,lat,lon,gps,zip_code
0,7706048,2017-01-06 09:51:00,PA,2178402.0,<NA>,3400 SPRUCE ST,BUS ONLY ZONE CC,76.0,PENN,39.95062073,-75.19395605,False,19104.0
1,7706049,2017-03-30 15:42:00,PA,4344539.0,<NA>,1700 MARKET ST,PARKING PROHBITED CC,51.0,PPA,39.954735,-75.170168,True,19103.0
2,7706050,2017-03-01 14:16:00,PA,510609.0,<NA>,1 S 18TH ST,TAXI STAND,31.0,PPA,39.952495,-75.169853,True,19103.0
3,7706051,2017-01-14 08:51:00,PA,2251747.0,1.0,200 ARCH ST,STOP PROHIBITED CC,76.0,PPA,39.95197551,-75.14362158,False,19106.0
4,7706052,2017-02-01 09:30:00,PA,4344540.0,<NA>,1700 COMMERCE,PARKING PROHBITED CC,51.0,CENTER C,<NA>,<NA>,<NA>,<NA>


In [5]:
parking = parking.dropna(subset=['lon', 'lat'])
parking.shape

(7661032, 13)

## Neighborhood Polygons

* First we read with Geopandas to look the data

In [6]:
neighborhoods = gpd.read_file("data/Neighborhoods/philly_neigh_wgs84.shp")
neighborhoods.head()

,NAME,LISTNAME,MAPNAME,Shape_Leng,Shape_Area,geometry
0,BRIDESBURG,Bridesburg,Bridesburg,27814.546521,4.458626e+07,"POLYGON ((-75.06773 40.00540, -75.06765 40.005..."
1,BUSTLETON,Bustleton,Bustleton,48868.458365,1.140504e+08,"POLYGON ((-75.01560 40.09487, -75.01769 40.092..."
2,CEDARBROOK,Cedarbrook,Cedarbrook,20021.415802,2.487174e+07,"POLYGON ((-75.18848 40.07273, -75.18846 40.072..."
3,CHESTNUT_HILL,Chestnut Hill,Chestnut Hill,56394.297195,7.966498e+07,"POLYGON ((-75.21221 40.08603, -75.21211 40.086..."
4,EAST_FALLS,East Falls,East Falls,27400.776417,4.057689e+07,"POLYGON ((-75.18479 40.02837, -75.18427 40.027..."


In [19]:
neighborhoods.tail()

,NAME,LISTNAME,MAPNAME,Shape_Leng,Shape_Area,geometry
153,PACKER_PARK,Packer Park,Packer Park,21816.017948,2.888897e+07,"POLYGON ((-75.18373 39.91351, -75.18194 39.913..."
154,PENNSPORT,Pennsport,Pennsport,11823.233108,6.492473e+06,"POLYGON ((-75.14654 39.93004, -75.14805 39.921..."
155,NEWBOLD,Newbold,Newbold,10052.570885,5.876853e+06,"POLYGON ((-75.16986 39.92312, -75.17015 39.921..."
156,WEST_PASSYUNK,West Passyunk,West Passyunk,10499.291848,6.494799e+06,"POLYGON ((-75.17630 39.92425, -75.17798 39.923..."
157,EAST_PASSYUNK,East Passyunk,East Passyunk,10987.761846,6.502097e+06,"POLYGON ((-75.15685 39.92897, -75.15712 39.927..."


In [7]:
neighborhoods.shape

(158, 6)

In [8]:
neighborhoods.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

* Cuspatial requires a specific format of input to carry out Points in Polygon (pip).

* Compute from a set of points and a set of polygons which points fall within which polygons. Note that polygons_(x,y) must be specified as closed polygons: the first and last coordinate of each polygon must be the same.



In [9]:
neighborhood_cuspatial = cuspatial.read_polygon_shapefile("data/Neighborhoods/philly_neigh_wgs84.shp")

In [10]:
neighborhood_cuspatial

(0        0
 1        1
 2        2
 3        3
 4        4
       ... 
 153    153
 154    154
 155    155
 156    156
 157    157
 Name: f_pos, Length: 158, dtype: int32,
 0          0
 1        280
 2        390
 3        472
 4        711
        ...  
 153    14579
 154    14617
 155    14633
 156    14650
 157    14674
 Name: r_pos, Length: 158, dtype: int32,
                x          y
 0     -75.067733  40.005401
 1     -75.068340  40.006924
 2     -75.068547  40.007323
 3     -75.068870  40.008550
 4     -75.068878  40.008580
 ...          ...        ...
 14691 -75.160758  39.925697
 14692 -75.159187  39.925487
 14693 -75.157677  39.925271
 14694 -75.157118  39.927752
 14695 -75.156845  39.928970
 
 [14696 rows x 2 columns])

In [11]:
neighborhood_cuspatial[0]

0        0
1        1
2        2
3        3
4        4
      ... 
153    153
154    154
155    155
156    156
157    157
Name: f_pos, Length: 158, dtype: int32

In [12]:
neighborhood_cuspatial[1]

0          0
1        280
2        390
3        472
4        711
       ...  
153    14579
154    14617
155    14633
156    14650
157    14674
Name: r_pos, Length: 158, dtype: int32

In [13]:
neighborhood_cuspatial[2]

,x,y
0,-75.067733,40.005401
1,-75.068340,40.006924
2,-75.068547,40.007323
3,-75.068870,40.008550
4,-75.068878,40.008580
...,...,...
14691,-75.160758,39.925697
14692,-75.159187,39.925487
14693,-75.157677,39.925271
14694,-75.157118,39.927752


In [14]:
def create_iterations(start, end, batches):
    iterations = list(np.arange(start, end, batches))
    iterations.append(end)
    return iterations

In [15]:
pip_iterations = create_iterations(0, 158, 31)

In [16]:
pip_iterations

[0, 31, 62, 93, 124, 155, 158]

*  The result of point_in_polygon is a DataFrame of Boolean values indicating whether each point (rows) falls within each polygon (columns).
* note input Series x and y will not be index aligned, but computed as sequential arrays.

In [17]:
def perform_pip(cuda_df, cuspatial_data, polygon_gdf_col, iter_batch):
    cuda_df['polygonID'] = " "
    cuda_df['polygonName'] = " "
    for i in range(len(iter_batch)-1):
        start = pip_iterations[i]
        end = pip_iterations[i+1]
        pip = cuspatial.point_in_polygon(cuda_df['lon'] , cuda_df['lat'], 
                                         cuspatial_data[0][start:end], 
                                         cuspatial_data[1], 
                                         cuspatial_data[2]['x'], 
                                         cuspatial_data[2]['y']
                                        )

        for i in pip.columns:
            cuda_df['polygonID'].loc[pip[i]] = i
            cuda_df['polygonName'].loc[pip[i]] = polygon_gdf_col[i]
    return cuda_df
    

In [18]:
%%time
sjoined_cudf = perform_pip(parking, neighborhood_cuspatial, neighborhoods["NAME"], pip_iterations)

CPU times: user 11.8 s, sys: 2.32 s, total: 14.1 s
Wall time: 14.2 s


In [21]:
sjoined_cudf.sample(10)

,anon_ticket_number,issue_datetime,state,anon_plate_id,division,location,violation_desc,fine,issuing_agency,lat,lon,gps,zip_code,polygonID,polygonName
3038031,4826123,2015-01-21 13:04:00,MN,157262.0,1.0,600 S 17TH ST,FIRE HYDRANT,76.0,PPA,39.944026,-75.170385,False,19146.0,101,GRADUATE_HOSPITAL
4202229,3697349,2014-03-10 14:06:00,PA,657051.0,1.0,1 S 33RD ST,METER EXPIRED CC,36.0,PPA,39.955207,-75.189547,False,19104.0,125,UNIVERSITY_CITY
4202182,3697269,2014-03-01 18:02:00,MD,130479.0,<NA>,1500 W PASSYUNK AVE,METER EXPIRED,26.0,PPA,39.925001,-75.172101,False,<NA>,155,NEWBOLD
3037992,4826082,2015-03-16 11:32:00,NJ,192916.0,1.0,700 SHIRLEY ST,OVER TIME LIMIT,26.0,PPA,39.967416,-75.167496,False,19130.0,96,FRANCISVILLE
7671165,162192,2012-05-21 12:19:00,PA,1654890.0,1.0,700 S 15TH ST,OVER TIME LIMIT,26.0,PPA,39.942844,-75.167404,False,19146.0,101,GRADUATE_HOSPITAL
7671136,162130,2012-04-19 10:30:00,CT,11533.0,1.0,700 SANSOM ST,METER EXPIRED CC,36.0,PPA,39.948685,-75.153305,False,19106.0,91,OLD_CITY
4211089,3716822,2014-02-26 18:28:00,PA,1669777.0,1.0,2300 N WATTS ST,SIDEWALK,51.0,PPA,39.988514,-75.154920,False,<NA>,84,HARTRANFT
3046836,4835051,2015-02-14 18:05:00,PA,1461813.0,1.0,4806 N BROAD ST,BUS ONLY ZONE,51.0,PPA,40.026892,-75.147388,False,19141.0,89,LOGAN
3046874,4835089,2015-01-16 22:20:00,PA,1440182.0,1.0,800 S 18TH ST,PARKING PROHBITED,41.0,PPA,39.941414,-75.172565,False,19146.0,101,GRADUATE_HOSPITAL
4211127,3716903,2014-02-24 09:03:00,PA,1428533.0,1.0,2900 N 5TH ST,METER EXPIRED,26.0,PPA,39.995914,-75.139360,False,19133.0,75,FAIRHILL


In [27]:
sjoined_cudf[sjoined_cudf["polygonID"] == "157"]


,anon_ticket_number,issue_datetime,state,anon_plate_id,division,location,violation_desc,fine,issuing_agency,lat,lon,gps,zip_code,polygonID,polygonName
21,7689058,2017-01-10 20:42:00,MA,4338404.0,1.0,900 MIFFLIN ST,OVER TIME LIMIT,26.0,PPA,39.925820,-75.161552,False,19148.0,157,EAST_PASSYUNK
102,7689139,2017-01-25 14:26:00,NJ,1796077.0,<NA>,1900 E PASSYUNK AVE,METER EXPIRED,26.0,PPA,39.925765,-75.168753,True,19148.0,157,EAST_PASSYUNK
103,7689140,2017-02-06 12:23:00,NJ,1796077.0,<NA>,1700 SBLK S 13TH ST,OVER TIME LIMIT,26.0,PPA,39.927888,-75.166788,True,<NA>,157,EAST_PASSYUNK
123,7689160,2017-01-10 15:50:00,PA,4338458.0,1.0,1700 E PASSYUNK AVE,METER EXPIRED,26.0,PPA,39.928369,-75.165258,False,19148.0,157,EAST_PASSYUNK
335,7689372,2017-03-11 11:01:00,PA,1834054.0,<NA>,1800 E PASSYUNK AVE,STOPPING PROHIBITED,51.0,PPA,39.926617,-75.167185,True,19148.0,157,EAST_PASSYUNK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8347207,1505074,2012-06-23 15:01:00,PA,723236.0,1.0,1700 E PASSYUNK AVE,METER EXPIRED,26.0,PPA,39.928369,-75.165258,False,19148.0,157,EAST_PASSYUNK
8347468,1505579,2012-04-02 10:07:00,PA,581091.0,1.0,1899 S 13TH ST,METER EXPIRED,26.0,PPA,39.926755,-75.167003,False,19148.0,157,EAST_PASSYUNK
8347469,1505580,2012-06-19 10:44:00,PA,581091.0,1.0,1899 S 13TH ST,METER EXPIRED,26.0,PPA,39.926755,-75.167003,False,19148.0,157,EAST_PASSYUNK
8347500,1505658,2012-03-27 12:44:00,PA,815003.0,1.0,1700 S 13TH ST,OVER TIME LIMIT,26.0,PPA,39.928679,-75.166580,False,19148.0,157,EAST_PASSYUNK
